# 전국의 치킨집 정보 표시 : 부어치킨

# module import
requests, selenium, BeautifulSoup : 크롤링 모듈
folium : 지도 모듈

In [49]:
import requests
import folium
from bs4 import BeautifulSoup as bs
import time 
import json

# 테스트 코드 작성(1페이지)

In [50]:
# 부어치킨 매장찾기 페이지
url = 'http://www.boor.co.kr/store/store.aspx' 
store_type = '%ub9e4%uc7a5%ucc3e%uae30'
requests.utils.unquote(store_type)
params = {
    'store_type': store_type
}
resp = requests.get(url, params=params)
soup = bs(resp.text, 'html.parser')
jijum_list = list(map(lambda i:i.text, soup.select('li.sto_title')))
addr_list = list(map(lambda i:i['addr'], soup.select('li.add_text[groupname=Address]')))
tel_list = list(map(lambda i:i.text, soup.select('li.add_text[groupname=Tel]')))
store_list = [(j, a, t) for j, a, t in zip(jijum_list, tel_list, addr_list )]


In [51]:
def getLatLng(addr):
#     addr = '서울특별시 구로구 디지털로 306'
    app_key = 'KakaoAK' + ' 78106cafe4d656639f171b177dda611f'
    url = 'https://dapi.kakao.com/v2/local/search/address.json'
    params = {
        'query': addr,
        'page': '',
        'AddressSize': ''
    }
    headers  = {
        'Authorization': app_key
    }
    resp = requests.get(url, params=params, headers=headers)
    if resp.json()['documents'] != []:
        json = resp.json()['documents'][0]
        return (json['x'], json['y'])
    else:
        fixed_addr = addr.replace(addr.split(' ')[-1], '').strip()
        if fixed_addr == '':
            print('불러오지 못했습니다 : ', addr)
            return (0, 0)
        else:
            return getLatLng(fixed_addr)
        
def get_avg_point(x_list, y_list):
    x = sum(x_list) / len(x_list)
    y = sum(y_list) / len(y_list)
    return x, y

def draw_map(addr):
    map_noman = folium.Map(location=get_avg_point(addr_point_list),zoom_start=7)
    disabled_sup = list(set(df_noman['장애인편의발급기형태']))
    for x, y in addr_point_list:
        if df_noman['장애인편의발급기형태'][cnt] == disabled_sup[0]:
            add_marker(map_noman, x, y)
    return map_noman

def draw_marker(x, y, tel):
    marker = folium.Marker((x, y), popup=tel)
    icon=folium.Icon(color='orange',icon='glyphicon glyphicon-cutlery')
    icon.add_to(marker)
    return marker

# requests 모듈 활용 크롤링

In [8]:
import requests
import folium
from bs4 import BeautifulSoup as bs
import time 
import json
from tinydb import TinyDB, Query

In [52]:
url = 'http://www.boor.co.kr/store/store.aspx'
cnt = 0
store_list = []
while True:
    cnt += 1
    params = {
        'page': cnt
    }
    resp = requests.get(url,params=params)
    print(resp.status_code)
    soup = bs(resp.text, 'html.parser')
    jijum_list = list(map(lambda i:i.text, soup.select('li.sto_title')))
    if jijum_list == []:
        break
    addr_list = list(map(lambda i:i['addr'], soup.select('ul.sto_text_box li')))
    tel_list = list(map(lambda i:i.text.replace('\n',''), soup.select('ul.sto_tel_box')))
    x_list = list(map(lambda i:i['lati'], soup.select('ul.sto_text_box li')))
    y_list = list(map(lambda i:i['longi'], soup.select('ul.sto_text_box li')))
    store_zip = zip(jijum_list, tel_list, addr_list, x_list, y_list)
    print(tel_list, addr_list, x_list, y_list)
    store_list.extend([{'jijum': j, 'tel': a, 'addr': t, 'x': x, 'y': y} for j, a, t, x, y in store_zip])
    print(store_list)
for store in store_list:
    if store['x'] == '' or store['y'] == '':
        store['y'], store['x'] = getLatLng(store['addr'])      
store_list


200
['031-877-9934', '032-571-8417', '032-583-6699', '032-576-0709', '032-425-2727', '042-525-9225'] ['경기도 의정부시 가능동 732-21', '인천광역시 서구 가좌동 142-36 부어치킨', '인천광역시 서구 가좌동 350-5', '인천광역시 서구 가좌동 30-25', '인천광역시 남동구 간석1동 285-7 오성상가 104호 부어치킨', '대전광역시 서구 갈마동 351-2 부어치킨'] ['37.7433970807', '37.4960848492', '37.4871429793', '37.4939085931', '37.4605487512', '36.3472713049'] ['127.035621563', '126.673722273', '126.685570175', '126.684546709', '126.707998161', '127.37171823']
[{'jijum': '부어치킨[가능점]', 'tel': '031-877-9934', 'addr': '경기도 의정부시 가능동 732-21', 'x': '37.7433970807', 'y': '127.035621563'}, {'jijum': '부어치킨[가좌1동점]', 'tel': '032-571-8417', 'addr': '인천광역시 서구 가좌동 142-36 부어치킨', 'x': '37.4960848492', 'y': '126.673722273'}, {'jijum': '부어치킨[가좌4동점]', 'tel': '032-583-6699', 'addr': '인천광역시 서구 가좌동 350-5', 'x': '37.4871429793', 'y': '126.685570175'}, {'jijum': '부어치킨[가좌동점]', 'tel': '032-576-0709', 'addr': '인천광역시 서구 가좌동 30-25', 'x': '37.4939085931', 'y': '126.684546709'}, {'jijum': '부어치킨[간석1동점]', 'tel': '03

['031-766-9941', '062-511-8280', '062-351-2898', '053-474-9595', '033-652-9582', '031-566-8687'] ['경기도 광주시 중대동 105-1 부어치킨', '광주광역시 북구 용전동 466', '광주광역시 서구 광천동 91-1 부어치킨', '대구광역시 남구 대명동 2008-11', '강원도 강릉시 교동 1888-5 하나로마트 내 부어치킨', '경기도 구리시 교문동 219-29 부어치킨'] ['37.4014974036', '35.2420620789', '35.1646120303', '35.8517145258', '37.763088149', '37.5991668279'] ['127.221839405', '126.896873753', '126.882553092', '128.59089432', '128.877227482', '127.133273857']
[{'jijum': '부어치킨[가능점]', 'tel': '031-877-9934', 'addr': '경기도 의정부시 가능동 732-21', 'x': '37.7433970807', 'y': '127.035621563'}, {'jijum': '부어치킨[가좌1동점]', 'tel': '032-571-8417', 'addr': '인천광역시 서구 가좌동 142-36 부어치킨', 'x': '37.4960848492', 'y': '126.673722273'}, {'jijum': '부어치킨[가좌4동점]', 'tel': '032-583-6699', 'addr': '인천광역시 서구 가좌동 350-5', 'x': '37.4871429793', 'y': '126.685570175'}, {'jijum': '부어치킨[가좌동점]', 'tel': '032-576-0709', 'addr': '인천광역시 서구 가좌동 30-25', 'x': '37.4939085931', 'y': '126.684546709'}, {'jijum': '부어치킨[간석1동점]', 'tel': '032-425-272

['02-452-9494', '032-551-8998', '053-558-9800', '031-853-9585', '031-322-8954', '031-988-3336'] ['서울특별시 광진구 군자동 146-10', '인천광역시 계양구 귤현동 507-12 부어치킨', '대구광역시 달서구 성당동 636-6', '경기도 의정부시 신곡동 757-2 부어치킨', '경기도 용인시 처인구 김량장동 63-9 용인에스원스마트빌', '경기도 김포시 감정동 510-5 , 101호'] ['37.55402901378991', '37.5673596161', '35.8432247669', '37.7486789757', '37.2341415411794', '37.6229281463'] ['127.07335304891505', '126.744898544', '128.55211155', '127.064372088', '127.209391465262', '126.697260045']
[{'jijum': '부어치킨[가능점]', 'tel': '031-877-9934', 'addr': '경기도 의정부시 가능동 732-21', 'x': '37.7433970807', 'y': '127.035621563'}, {'jijum': '부어치킨[가좌1동점]', 'tel': '032-571-8417', 'addr': '인천광역시 서구 가좌동 142-36 부어치킨', 'x': '37.4960848492', 'y': '126.673722273'}, {'jijum': '부어치킨[가좌4동점]', 'tel': '032-583-6699', 'addr': '인천광역시 서구 가좌동 350-5', 'x': '37.4871429793', 'y': '126.685570175'}, {'jijum': '부어치킨[가좌동점]', 'tel': '032-576-0709', 'addr': '인천광역시 서구 가좌동 30-25', 'x': '37.4939085931', 'y': '126.684546709'}, {'jijum': '부어치킨[간석1동

200
['031-573-8441', '053-556-8292', '043-215-1239', '031-835-2565', '033-641-9939', '041-662-3611'] ['경기도 남양주시 진접읍 내각리 322-4 부어치킨', '대구광역시 서구 내당동 355-7', '충청북도 청주시 청원구 내수읍 마산리 197-194', '경기도 연천군 백학면 노곡리 727-3 부어치킨', '강원도 강릉시 노암동 613-7 부어치킨', '충청남도 서산시 지곡면 무장리 920 늘푸른오스카빌'] ['37.7046668946', '35.8589114306', '36.7217867875', '37.9991624929', '37.7428252875', '36.5023898238'] ['127.165618169', '128.551040253', '127.537283696', '126.920795566', '128.893731809', '126.62576908']
[{'jijum': '부어치킨[가능점]', 'tel': '031-877-9934', 'addr': '경기도 의정부시 가능동 732-21', 'x': '37.7433970807', 'y': '127.035621563'}, {'jijum': '부어치킨[가좌1동점]', 'tel': '032-571-8417', 'addr': '인천광역시 서구 가좌동 142-36 부어치킨', 'x': '37.4960848492', 'y': '126.673722273'}, {'jijum': '부어치킨[가좌4동점]', 'tel': '032-583-6699', 'addr': '인천광역시 서구 가좌동 350-5', 'x': '37.4871429793', 'y': '126.685570175'}, {'jijum': '부어치킨[가좌동점]', 'tel': '032-576-0709', 'addr': '인천광역시 서구 가좌동 30-25', 'x': '37.4939085931', 'y': '126.684546709'}, {'jijum': '부어치킨[간석1동점]'

['053-353-3526', '053-633-9285', '02-826-9595', '053-565-0099', '054-282-3999', '031-919-8592'] ['대구광역시 북구 침산동 363-6 현대그린상가 103호', '대구광역시 달성군 화원읍 천내리 53-14', '서울특별시 동작구 대방동 23-58 부어치킨', '대구광역시 서구 비산동 286-10 1층 상가', '경상북도 포항시 남구 대잠동 921-10', '경기도 고양시 일산서구 대화동 863-3 부어치킨'] ['35.8900963079', '35.8054266928', '37.5115879567', '35.8674612549', '36.0156558894', '37.6708006689'] ['128.587059481', '128.503045485', '126.932158364', '128.571582211', '129.34898786', '126.735399952']
[{'jijum': '부어치킨[가능점]', 'tel': '031-877-9934', 'addr': '경기도 의정부시 가능동 732-21', 'x': '37.7433970807', 'y': '127.035621563'}, {'jijum': '부어치킨[가좌1동점]', 'tel': '032-571-8417', 'addr': '인천광역시 서구 가좌동 142-36 부어치킨', 'x': '37.4960848492', 'y': '126.673722273'}, {'jijum': '부어치킨[가좌4동점]', 'tel': '032-583-6699', 'addr': '인천광역시 서구 가좌동 350-5', 'x': '37.4871429793', 'y': '126.685570175'}, {'jijum': '부어치킨[가좌동점]', 'tel': '032-576-0709', 'addr': '인천광역시 서구 가좌동 30-25', 'x': '37.4939085931', 'y': '126.684546709'}, {'jijum': '부어치킨[간석1동점]', '

...

['063-212-9288', '031-594-9282', '031-611-7498', '031-618-5668', '031-658-7279', '031-658-0096'] ['전라북도 전주시 덕진구 팔복동2가 247-13 부어치킨', '경기도 남양주시 평내동 604 부어치킨', '경기도 평택시 서정동 844-9', '경기도 평택시 소사동 0 평택효성해링턴플레이스2단지아파트그린생활시설 2동 102호', '경기도 평택시 비전동 879-5 비전현대아파트', '경기도 평택시 죽백동 816 평택 소사벌 우미린 레이크파크'] ['35.8529112561', '37.643289491', '37.066273694796955', '36.98869540122556', '36.99377120956189', '37.003278335'] ['127.105802185', '127.236937298', '127.06036811647643', '127.12751942536447', '127.11609663031274', '127.124294628']
[{'jijum': '부어치킨[가능점]', 'tel': '031-877-9934', 'addr': '경기도 의정부시 가능동 732-21', 'x': '37.7433970807', 'y': '127.035621563'}, {'jijum': '부어치킨[가좌1동점]', 'tel': '032-571-8417', 'addr': '인천광역시 서구 가좌동 142-36 부어치킨', 'x': '37.4960848492', 'y': '126.673722273'}, {'jijum': '부어치킨[가좌4동점]', 'tel': '032-583-6699', 'addr': '인천광역시 서구 가좌동 350-5', 'x': '37.4871429793', 'y': '126.685570175'}, {'jijum': '부어치킨[가좌동점]', 'tel': '032-576-0709', 'addr': '인천광역시 서구 가좌동 30-25', 'x': '37.4939085931', 'y

['031-651-3308', '031-535-8450', '031-314-6545', '043-653-4646', '053-852-9285', '032-866-9292'] ['경기도 평택시 평택동 45-8', '경기도 포천시 신읍동 201-13, 1층', '경기도 시흥시 하상동 375-1', '충청북도 제천시 하소동 362 부어치킨', '경상북도 경산시 하양읍 금락리 130-5 하양2차우방타운 206동 106호', '인천광역시 남구 학익동 721 풍림아이원상가 105호 부어치킨'] ['36.9922715204', '37.899046814', '37.3914693768', '37.1437240701', '35.9089203043', '37.4411096475'] ['127.086928107', '127.204678018', '126.812727495', '128.198303204', '128.812057139', '126.658638265']
[{'jijum': '부어치킨[가능점]', 'tel': '031-877-9934', 'addr': '경기도 의정부시 가능동 732-21', 'x': '37.7433970807', 'y': '127.035621563'}, {'jijum': '부어치킨[가좌1동점]', 'tel': '032-571-8417', 'addr': '인천광역시 서구 가좌동 142-36 부어치킨', 'x': '37.4960848492', 'y': '126.673722273'}, {'jijum': '부어치킨[가좌4동점]', 'tel': '032-583-6699', 'addr': '인천광역시 서구 가좌동 350-5', 'x': '37.4871429793', 'y': '126.685570175'}, {'jijum': '부어치킨[가좌동점]', 'tel': '032-576-0709', 'addr': '인천광역시 서구 가좌동 30-25', 'x': '37.4939085931', 'y': '126.684546709'}, {'jijum': '부어치킨[간석1동점]', 

['043-855-1533', '044-865-8992', '031-972-9270', '053-611-9946', '032-429-0984', '031-591-7706'] ['충청북도 충주시 대소원면 만정리 194-6', '세종특별자치시 한솔동 968 공원빌딩', '경기도 고양시 덕양구 행신동 619-14 부어치킨', '대구광역시 달성군 현풍면 부리 453-6', '인천광역시 남동구 논현동 635-1 주공3단지 상가 105호 부어치킨', '경기도 남양주시 호평동 660-2 CH리베로3 101호'] ['36.96349166160774', '36.47994076658709', '37.6194603668', '35.697172221', '37.4031411595', '37.65411348051775'] ['127.87113191039549', '127.25342833067646', '126.832527629', '128.443439267', '126.712978257', '127.2485170009729']
[{'jijum': '부어치킨[가능점]', 'tel': '031-877-9934', 'addr': '경기도 의정부시 가능동 732-21', 'x': '37.7433970807', 'y': '127.035621563'}, {'jijum': '부어치킨[가좌1동점]', 'tel': '032-571-8417', 'addr': '인천광역시 서구 가좌동 142-36 부어치킨', 'x': '37.4960848492', 'y': '126.673722273'}, {'jijum': '부어치킨[가좌4동점]', 'tel': '032-583-6699', 'addr': '인천광역시 서구 가좌동 350-5', 'x': '37.4871429793', 'y': '126.685570175'}, {'jijum': '부어치킨[가좌동점]', 'tel': '032-576-0709', 'addr': '인천광역시 서구 가좌동 30-25', 'x': '37.4939085931', 'y': '126.684

200
['032-569-2530', '032-548-5918', '063-221-9209', '02-777-9512', '033-243-9292', '02-3280-1577'] ['인천광역시 계양구 효성동 200-1 현대아파트 상가 101호', '인천광역시 계양구 효성동 28 부어치킨', '전라북도 전주시 완산구 효자동1가 373 뷰티빌 상가 105호', '서울특별시 용산구 후암동 415-37 부어치킨', '강원도 춘천시 후평동 67-52 1층 102호', '서울특별시 동작구 흑석동 8-32'] ['37.526155139', '37.5352618069', '35.8070425509', '37.5465649117', '37.8763877423', '37.5077242143'] ['126.717915167', '126.710214334', '127.115628145', '126.981867253', '127.746626178', '126.964612874']
[{'jijum': '부어치킨[가능점]', 'tel': '031-877-9934', 'addr': '경기도 의정부시 가능동 732-21', 'x': '37.7433970807', 'y': '127.035621563'}, {'jijum': '부어치킨[가좌1동점]', 'tel': '032-571-8417', 'addr': '인천광역시 서구 가좌동 142-36 부어치킨', 'x': '37.4960848492', 'y': '126.673722273'}, {'jijum': '부어치킨[가좌4동점]', 'tel': '032-583-6699', 'addr': '인천광역시 서구 가좌동 350-5', 'x': '37.4871429793', 'y': '126.685570175'}, {'jijum': '부어치킨[가좌동점]', 'tel': '032-576-0709', 'addr': '인천광역시 서구 가좌동 30-25', 'x': '37.4939085931', 'y': '126.684546709'}, {'jijum': '부어치킨[간

[{'jijum': '부어치킨[가능점]',
  'tel': '031-877-9934',
  'addr': '경기도 의정부시 가능동 732-21',
  'x': '37.7433970807',
  'y': '127.035621563'},
 {'jijum': '부어치킨[가좌1동점]',
  'tel': '032-571-8417',
  'addr': '인천광역시 서구 가좌동 142-36 부어치킨',
  'x': '37.4960848492',
  'y': '126.673722273'},
 {'jijum': '부어치킨[가좌4동점]',
  'tel': '032-583-6699',
  'addr': '인천광역시 서구 가좌동 350-5',
  'x': '37.4871429793',
  'y': '126.685570175'},
 {'jijum': '부어치킨[가좌동점]',
  'tel': '032-576-0709',
  'addr': '인천광역시 서구 가좌동 30-25',
  'x': '37.4939085931',
  'y': '126.684546709'},
 {'jijum': '부어치킨[간석1동점]',
  'tel': '032-425-2727',
  'addr': '인천광역시 남동구 간석1동 285-7 오성상가 104호 부어치킨',
  'x': '37.4605487512',
  'y': '126.707998161'},
 {'jijum': '부어치킨[갈마점_대전지사]',
  'tel': '042-525-9225',
  'addr': '대전광역시 서구 갈마동 351-2 부어치킨',
  'x': '36.3472713049',
  'y': '127.37171823'},
 {'jijum': '부어치킨[갈현2호점]',
  'tel': '02-388-3102',
  'addr': '서울특별시 은평구 갈현동 527-15 부어치킨',
  'x': '37.6146903324',
  'y': '126.909960389'},
 {'jijum': '부어치킨[갈현점]',
  'tel': '02-386-8

# 지도에 표시

In [53]:
x_list = list(map(lambda store:float(store['x']),store_list))
y_list = list(map(lambda store:float(store['y']),store_list))
tel_list = list(map(lambda store:store['tel'],store_list))
map_boor = folium.Map(location=get_avg_point(x_list,y_list), zoom_start=7)
for x, y, tel in zip(x_list,y_list, tel_list):
    draw_marker(x, y, tel).add_to(map_boor)
map_boor

# DB에 저장

In [54]:
db = TinyDB('boorchicken')

In [55]:
for store in store_list:
    db.insert(store)
db.all()

[{'jijum': '부어치킨[가능점]',
  'tel': '031-877-9934',
  'addr': '경기도 의정부시 가능동 732-21',
  'x': '37.7433970807',
  'y': '127.035621563'},
 {'jijum': '부어치킨[가좌1동점]',
  'tel': '032-571-8417',
  'addr': '인천광역시 서구 가좌동 142-36 부어치킨',
  'x': '37.4960848492',
  'y': '126.673722273'},
 {'jijum': '부어치킨[가좌4동점]',
  'tel': '032-583-6699',
  'addr': '인천광역시 서구 가좌동 350-5',
  'x': '37.4871429793',
  'y': '126.685570175'},
 {'jijum': '부어치킨[가좌동점]',
  'tel': '032-576-0709',
  'addr': '인천광역시 서구 가좌동 30-25',
  'x': '37.4939085931',
  'y': '126.684546709'},
 {'jijum': '부어치킨[간석1동점]',
  'tel': '032-425-2727',
  'addr': '인천광역시 남동구 간석1동 285-7 오성상가 104호 부어치킨',
  'x': '37.4605487512',
  'y': '126.707998161'},
 {'jijum': '부어치킨[갈마점_대전지사]',
  'tel': '042-525-9225',
  'addr': '대전광역시 서구 갈마동 351-2 부어치킨',
  'x': '36.3472713049',
  'y': '127.37171823'},
 {'jijum': '부어치킨[갈현2호점]',
  'tel': '02-388-3102',
  'addr': '서울특별시 은평구 갈현동 527-15 부어치킨',
  'x': '37.6146903324',
  'y': '126.909960389'},
 {'jijum': '부어치킨[갈현점]',
  'tel': '02-386-8